In [ ]:
pip install altair vega_datasets geopandas jsonschema==3.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import altair as alt
import geopandas as gpd
from IPython.display import display, HTML

In [ ]:
# v for visits
v = pd.read_csv(r'monthly_vis_by_site_zeroed.csv')
v.head()

data_geojson = gpd.read_file('EagleCreek_AOI.geojson')
data_geojson.head()

geo_vis = data_geojson.merge(v, on=['siteid', 'Name'])
geo_vis

,descriptio,layer,path,Name,siteid,geometry,month,year,years_fire,pre_post_fire,burned,closed_ever,tot_vis,closed_month,d2p
0,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2015,-3,pre,yes,yes,169,open,2015-01-15
1,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2016,-2,pre,yes,yes,436,open,2016-01-15
2,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2017,-1,pre,yes,yes,788,open,2017-01-15
3,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2018,0,post,yes,yes,0,closed,2018-01-15
4,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2019,1,post,yes,yes,0,closed,2019-01-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,Sama added Nov 2019. Edited 20200714 to remove...,Eagle_ creek_substitutes_111319 Open,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Larch Mountain,CRG_046,"MULTIPOLYGON Z (((-122.11398 45.57524 0.00000,...",12,2014,-3,pre,no,yes,43,open,2014-12-15
2497,Sama added Nov 2019. Edited 20200714 to remove...,Eagle_ creek_substitutes_111319 Open,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Larch Mountain,CRG_046,"MULTIPOLYGON Z (((-122.11398 45.57524 0.00000,...",12,2015,-2,pre,no,yes,81,open,2015-12-15
2498,Sama added Nov 2019. Edited 20200714 to remove...,Eagle_ creek_substitutes_111319 Open,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Larch Mountain,CRG_046,"MULTIPOLYGON Z (((-122.11398 45.57524 0.00000,...",12,2016,-1,pre,no,yes,128,open,2016-12-15
2499,Sama added Nov 2019. Edited 20200714 to remove...,Eagle_ creek_substitutes_111319 Open,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Larch Mountain,CRG_046,"MULTIPOLYGON Z (((-122.11398 45.57524 0.00000,...",12,2017,0,post,no,yes,0,closed,2017-12-15


Total visitors summed across trails for each year to show seasonality.

In [ ]:
total_vis = v.groupby(['d2p', 'month', 'year'], as_index=False)['tot_vis'].sum()

alt.Chart(total_vis).mark_line().encode(
    x=alt.X('monthdate(d2p):Q', title = 'Month of the year'),
    y=alt.Y('tot_vis:Q', title = 'Total visitors across all trails'),
    #facet=alt.Facet('year'),
)

alt.Chart(...)

In [ ]:
# slider to choose top N in popularity
slider = alt.binding_range(min=1, max=41, step=1, name="Show most popular N trails")
topN = alt.selection_single(name='topN', fields=['cutoff'], bind=slider, init={'cutoff': 15})

nearest = alt.selection_single(nearest=True, on='mouseover', encodings=['y'])

alt.Chart(geo_vis).mark_line().encode(
    x = alt.X('d2p:T', axis=alt.Axis(format='%B %Y',formatType='time',tickCount=alt.TimeIntervalStep(interval=alt.TimeInterval('month'),step=6))),
    y='tot_vis',
    color='Name',
    strokeWidth=alt.value(5),
    tooltip=['Name', 'pre_fire_rank', 'yearmonth(d2p)']
).transform_filter(
    alt.datum.pre_fire_rank <= topN.cutoff
).add_selection(
    topN
).properties(
    width=800
)

ValueError: ignored

alt.Chart(...)

Overlay of yearly trends of total visitors across trails; note the incomplete plots for years 2014 and 2019.

In [ ]:
alt.Chart(total_vis).mark_line().encode(
    x=alt.X('month:Q', title = 'Month of the year'),
    y=alt.Y('tot_vis:Q', title = 'Total visitors across all trails'),
    color = alt.Color('year:O', scale=alt.Scale(scheme='brownbluegreen'))
)

alt.Chart(...)

Exploring seasonal (weather) effects

In [ ]:
import numpy as np 

monthNames = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], dtype=object)
winter = np.array(['Dec', 'Jan', 'Feb'], dtype=object)
spring = np.array(['Mar', 'Apr', 'May'], dtype=object)
summer = np.array(['Jun', 'Jul', 'Aug'], dtype=object)
fall = np.array(['Sep', 'Oct', 'Nov'], dtype=object)

winter = np.array([12,1,2])

winter_vis = total_vis[(total_vis.month == 12) | (total_vis.month == 1) | (total_vis.month == 2)]
spring_vis = total_vis[(total_vis.month == 3) | (total_vis.month == 4) | (total_vis.month == 5)]
summer_vis = total_vis[(total_vis.month == 6) | (total_vis.month == 7) | (total_vis.month == 8)]
fall_vis = total_vis[(total_vis.month == 9) | (total_vis.month == 10) | (total_vis.month == 11)]


In [ ]:
winter = alt.Chart(winter_vis).mark_bar().encode(
    x=alt.X('month:N', title = 'Month of the year', sort = winter),
    y=alt.Y('tot_vis:Q', title = 'Total visitors across all trails'),
    color = alt.Color('year:O', scale=alt.Scale(scheme='brownbluegreen'))
).properties(
    title='Winter'
)

spring = alt.Chart(spring_vis).mark_bar().encode(
    x=alt.X('month:N', title = 'Month of the year'),
    y=alt.Y('tot_vis:Q', title = 'Total visitors across all trails'),
    color = alt.Color('year:O', scale=alt.Scale(scheme='brownbluegreen'))
).properties(
    title='Spring'
)

summer = alt.Chart(summer_vis).mark_bar().encode(
    x=alt.X('month:N', title = 'Month of the year'),
    y=alt.Y('tot_vis:Q', title = 'Total visitors across all trails'),
    color = alt.Color('year:O', scale=alt.Scale(scheme='brownbluegreen'))
).properties(
    title='Summer'
)

fall = alt.Chart(fall_vis).mark_bar().encode(
    x=alt.X('month:N', title = 'Month of the year'),
    y=alt.Y('tot_vis:Q', title = 'Total visitors across all trails'),
    color = alt.Color('year:O', scale=alt.Scale(scheme='brownbluegreen'))
).properties(
    title='Fall'
)

winter | spring | summer | fall

alt.HConcatChart(...)

In [ ]:
alt.Chart(v).mark_line(point = True).encode(
    x = alt.X("d2p:O", timeUnit="yearmonth", title="date"),
    y="tot_vis:O",
    color=alt.Color("siteid:N")
).transform_window(
    rank="rank()",
    sort=[alt.SortField("tot_vis", order="ascending")],
    groupby=["d2p"]
).properties(
    title="Trail Popularity Over Time",
    width=600,
    height=150,
)

alt.Chart(...)

Trail rank 3 years pre-fire

In [ ]:
popularity2014 = v[(v['year'] > 2013) & (v['year'] <= 2014)].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
popularity2014 = popularity2014.sort_values(by='tot_vis', ascending=False)
poprank2014 = popularity2014['Name'].tolist()

poprank2014

['Horsetail Falls',
 'Latourell Falls',
 'Oneonta Gorge',
 'Angels rest trail',
 'Eagle Creek',
 'Munra Falls',
 'Wahkeena Falls',
 'Beacon Rock',
 'Nesmith Point, McCord, Elowa Falls',
 'Cape Horn',
 'Washougal River area',
 'Tamanawas Falls trail',
 'Surveyors ridge/cooks meadow',
 'Umbrella/Sahalie Falls',
 'Larch Mountain',
 'Mitchell/Wygant area',
 'Mosier Twin Tunnels Trail',
 'Falls Creek Falls',
 'Herman Creek',
 'Dog Mountain/Augspurger Mtn',
 'Coyote Wall area',
 'Catherine Creek area',
 'Wind Mountain',
 'Mosier Plateau FCG trail',
 'Panther Creek Falls',
 'Ruckel Creek',
 'Klickitat Trail',
 'Starvation Creek area',
 'Tom McCall TNC trail',
 'Tanner Butte',
 'Indian Springs',
 'Dog River',
 'Crawford Oaks WDNR',
 'Lower Deschutes',
 'Spirit falls',
 'Memaloose trail',
 'Dog Creek Falls',
 'Horsetail Creek',
 'Cherry Orchard FCG trail',
 'Weldon Wagon Rd Trail',
 'Eight Mile trail']

Trail rank 2015

In [ ]:
popularity2015 = v[(v['year'] > 2014) & (v['year'] <= 2015)].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
popularity2015 = popularity2015.sort_values(by='tot_vis', ascending=False)
poprank2015 = popularity2015['Name'].tolist()

poprank2015

['Eagle Creek',
 'Oneonta Gorge',
 'Munra Falls',
 'Angels rest trail',
 'Horsetail Falls',
 'Latourell Falls',
 'Wahkeena Falls',
 'Beacon Rock',
 'Nesmith Point, McCord, Elowa Falls',
 'Umbrella/Sahalie Falls',
 'Washougal River area',
 'Cape Horn',
 'Tamanawas Falls trail',
 'Surveyors ridge/cooks meadow',
 'Falls Creek Falls',
 'Larch Mountain',
 'Dog Mountain/Augspurger Mtn',
 'Mitchell/Wygant area',
 'Coyote Wall area',
 'Catherine Creek area',
 'Tom McCall TNC trail',
 'Herman Creek',
 'Panther Creek Falls',
 'Starvation Creek area',
 'Mosier Twin Tunnels Trail',
 'Mosier Plateau FCG trail',
 'Wind Mountain',
 'Ruckel Creek',
 'Horsetail Creek',
 'Klickitat Trail',
 'Tanner Butte',
 'Spirit falls',
 'Dog Creek Falls',
 'Indian Springs',
 'Dog River',
 'Crawford Oaks WDNR',
 'Eight Mile trail',
 'Lower Deschutes',
 'Weldon Wagon Rd Trail',
 'Memaloose trail',
 'Cherry Orchard FCG trail']

Trail rank 2016

In [ ]:
popularity2016 = v[(v['year'] > 2015) & (v['year'] <= 2016)].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
popularity2016 = popularity2016.sort_values(by='tot_vis', ascending=False)
poprank2016 = popularity2016['Name'].tolist()

poprank2016

['Oneonta Gorge',
 'Munra Falls',
 'Eagle Creek',
 'Latourell Falls',
 'Horsetail Falls',
 'Wahkeena Falls',
 'Angels rest trail',
 'Beacon Rock',
 'Nesmith Point, McCord, Elowa Falls',
 'Umbrella/Sahalie Falls',
 'Tamanawas Falls trail',
 'Dog Mountain/Augspurger Mtn',
 'Surveyors ridge/cooks meadow',
 'Falls Creek Falls',
 'Cape Horn',
 'Washougal River area',
 'Larch Mountain',
 'Panther Creek Falls',
 'Tom McCall TNC trail',
 'Coyote Wall area',
 'Catherine Creek area',
 'Mitchell/Wygant area',
 'Wind Mountain',
 'Herman Creek',
 'Mosier Twin Tunnels Trail',
 'Starvation Creek area',
 'Mosier Plateau FCG trail',
 'Spirit falls',
 'Crawford Oaks WDNR',
 'Horsetail Creek',
 'Tanner Butte',
 'Ruckel Creek',
 'Dog Creek Falls',
 'Klickitat Trail',
 'Dog River',
 'Eight Mile trail',
 'Indian Springs',
 'Weldon Wagon Rd Trail',
 'Lower Deschutes',
 'Memaloose trail',
 'Cherry Orchard FCG trail']

Trail rank 2017

In [ ]:
popularity2017 = v[(v['year'] > 2016) & (v['year'] <= 2017)].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
popularity2017 = popularity2017.sort_values(by='tot_vis', ascending=False)
poprank2017 = popularity2017['Name'].tolist()

poprank2017

['Oneonta Gorge',
 'Munra Falls',
 'Latourell Falls',
 'Angels rest trail',
 'Dog Mountain/Augspurger Mtn',
 'Eagle Creek',
 'Horsetail Falls',
 'Beacon Rock',
 'Wahkeena Falls',
 'Umbrella/Sahalie Falls',
 'Nesmith Point, McCord, Elowa Falls',
 'Tamanawas Falls trail',
 'Cape Horn',
 'Surveyors ridge/cooks meadow',
 'Falls Creek Falls',
 'Tom McCall TNC trail',
 'Washougal River area',
 'Wind Mountain',
 'Mitchell/Wygant area',
 'Larch Mountain',
 'Spirit falls',
 'Coyote Wall area',
 'Mosier Plateau FCG trail',
 'Mosier Twin Tunnels Trail',
 'Herman Creek',
 'Ruckel Creek',
 'Catherine Creek area',
 'Starvation Creek area',
 'Eight Mile trail',
 'Horsetail Creek',
 'Dog Creek Falls',
 'Dog River',
 'Weldon Wagon Rd Trail',
 'Panther Creek Falls',
 'Indian Springs',
 'Tanner Butte',
 'Crawford Oaks WDNR',
 'Cherry Orchard FCG trail',
 'Lower Deschutes',
 'Klickitat Trail',
 'Memaloose trail']

Trail rank 2018

In [ ]:
popularity2018 = v[(v['year'] > 2017) & (v['year'] <= 2018)].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
popularity2018 = popularity2018.sort_values(by='tot_vis', ascending=False)
poprank2018 = popularity2018['Name'].tolist()

poprank2018

['Latourell Falls',
 'Beacon Rock',
 'Dog Mountain/Augspurger Mtn',
 'Umbrella/Sahalie Falls',
 'Falls Creek Falls',
 'Tamanawas Falls trail',
 'Tom McCall TNC trail',
 'Cape Horn',
 'Surveyors ridge/cooks meadow',
 'Washougal River area',
 'Wind Mountain',
 'Mitchell/Wygant area',
 'Coyote Wall area',
 'Spirit falls',
 'Herman Creek',
 'Mosier Twin Tunnels Trail',
 'Starvation Creek area',
 'Dog Creek Falls',
 'Weldon Wagon Rd Trail',
 'Angels rest trail',
 'Cherry Orchard FCG trail',
 'Eight Mile trail',
 'Catherine Creek area',
 'Mosier Plateau FCG trail',
 'Panther Creek Falls',
 'Dog River',
 'Crawford Oaks WDNR',
 'Klickitat Trail',
 'Lower Deschutes',
 'Memaloose trail',
 'Wahkeena Falls',
 'Horsetail Falls',
 'Larch Mountain',
 'Horsetail Creek',
 'Eagle Creek',
 'Tanner Butte',
 'Ruckel Creek',
 'Oneonta Gorge',
 'Nesmith Point, McCord, Elowa Falls',
 'Munra Falls',
 'Indian Springs']

Trail rank 2019

In [ ]:
popularity2019 = v[(v['year'] > 2018) & (v['year'] <= 2019)].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
popularity2019 = popularity2019.sort_values(by='tot_vis', ascending=False)
poprank2019 = popularity2019['Name'].tolist()

poprank2019

['Angels rest trail',
 'Latourell Falls',
 'Beacon Rock',
 'Dog Mountain/Augspurger Mtn',
 'Wahkeena Falls',
 'Tamanawas Falls trail',
 'Tom McCall TNC trail',
 'Horsetail Falls',
 'Umbrella/Sahalie Falls',
 'Falls Creek Falls',
 'Cape Horn',
 'Coyote Wall area',
 'Surveyors ridge/cooks meadow',
 'Washougal River area',
 'Larch Mountain',
 'Mitchell/Wygant area',
 'Wind Mountain',
 'Spirit falls',
 'Herman Creek',
 'Starvation Creek area',
 'Munra Falls',
 'Mosier Twin Tunnels Trail',
 'Panther Creek Falls',
 'Crawford Oaks WDNR',
 'Catherine Creek area',
 'Cherry Orchard FCG trail',
 'Eight Mile trail',
 'Klickitat Trail',
 'Dog Creek Falls',
 'Weldon Wagon Rd Trail',
 'Mosier Plateau FCG trail',
 'Dog River',
 'Memaloose trail',
 'Lower Deschutes',
 'Tanner Butte',
 'Eagle Creek',
 'Indian Springs',
 'Oneonta Gorge',
 'Nesmith Point, McCord, Elowa Falls',
 'Horsetail Creek',
 'Ruckel Creek']

In [ ]:
colorCustom = ['#2ca02c', '#ff7f0e']

alt.Chart(popularity2014[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = 'Trail name', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors'),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2014 Trail Rank',
    width=200,
    height=200,
)



alt.Chart(...)

In [ ]:
chart2014 = alt.Chart(popularity2014[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors'),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2014 Trail Rank (Sep-Dec)',
    width=200,
    height=200,
)

chart2015 = alt.Chart(popularity2015[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors'),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2015 Trail Rank (Jan-Dec)',
    width=200,
    height=200,
)

chart2016 = alt.Chart(popularity2016[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors'),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2016 Trail Rank (Jan-Dec)',
    width=200,
    height=200,
)

chart2017 = alt.Chart(popularity2017[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors'),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2017 Trail Rank (Jan-Dec)',
    width=200,
    height=200,
)

chart2018 = alt.Chart(popularity2018[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors'),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2018 Trail Rank (Jan-Dec)',
    width=200,
    height=200,
)

chart2019 = alt.Chart(popularity2019[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors'),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2019 Trail Rank (Jan-Sep)',
    width=200,
    height=200,
)

trail_popularity = chart2014 & chart2017 | chart2015 & chart2018 | chart2016 & chart2019
trail_popularity

alt.HConcatChart(...)

In [ ]:
trail_popularity.save('trail_popularity.html')

Trail popularity 3 years pre-fire (Sept 2014 - Sept 2015)

In [ ]:
start_date = '2014-09-15'
end_date = '2015-08-15'

test_n3 = v[ (v['d2p'] >= start_date) & (v['d2p'] <= end_date) ].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
test_n3 = test_n3.sort_values(by='tot_vis', ascending=False)
test_n3rank = test_n3['Name'].tolist()

test_n3rank

['Eagle Creek',
 'Oneonta Gorge',
 'Munra Falls',
 'Horsetail Falls',
 'Angels rest trail',
 'Latourell Falls',
 'Wahkeena Falls',
 'Beacon Rock',
 'Nesmith Point, McCord, Elowa Falls',
 'Washougal River area',
 'Umbrella/Sahalie Falls',
 'Cape Horn',
 'Tamanawas Falls trail',
 'Falls Creek Falls',
 'Surveyors ridge/cooks meadow',
 'Larch Mountain',
 'Dog Mountain/Augspurger Mtn',
 'Coyote Wall area',
 'Mitchell/Wygant area',
 'Catherine Creek area',
 'Tom McCall TNC trail',
 'Herman Creek',
 'Panther Creek Falls',
 'Mosier Twin Tunnels Trail',
 'Mosier Plateau FCG trail',
 'Starvation Creek area',
 'Wind Mountain',
 'Horsetail Creek',
 'Ruckel Creek',
 'Klickitat Trail',
 'Tanner Butte',
 'Spirit falls',
 'Dog Creek Falls',
 'Indian Springs',
 'Crawford Oaks WDNR',
 'Dog River',
 'Lower Deschutes',
 'Eight Mile trail',
 'Weldon Wagon Rd Trail',
 'Memaloose trail',
 'Cherry Orchard FCG trail']

Trail popularity 2 years pre-fire (Sept 2015 - Sept 2016)

In [ ]:
start_date = '2015-09-15'
end_date = '2016-08-15'

test_n2 = v[ (v['d2p'] >= start_date) & (v['d2p'] <= end_date) ].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
test_n2 = test_n2.sort_values(by='tot_vis', ascending=False)
test_n2rank = test_n2['Name'].tolist()

test_n2rank

['Oneonta Gorge',
 'Eagle Creek',
 'Munra Falls',
 'Latourell Falls',
 'Horsetail Falls',
 'Angels rest trail',
 'Wahkeena Falls',
 'Beacon Rock',
 'Nesmith Point, McCord, Elowa Falls',
 'Umbrella/Sahalie Falls',
 'Tamanawas Falls trail',
 'Dog Mountain/Augspurger Mtn',
 'Falls Creek Falls',
 'Surveyors ridge/cooks meadow',
 'Cape Horn',
 'Washougal River area',
 'Larch Mountain',
 'Panther Creek Falls',
 'Tom McCall TNC trail',
 'Catherine Creek area',
 'Mitchell/Wygant area',
 'Wind Mountain',
 'Coyote Wall area',
 'Herman Creek',
 'Mosier Twin Tunnels Trail',
 'Mosier Plateau FCG trail',
 'Starvation Creek area',
 'Spirit falls',
 'Crawford Oaks WDNR',
 'Horsetail Creek',
 'Ruckel Creek',
 'Tanner Butte',
 'Dog Creek Falls',
 'Eight Mile trail',
 'Dog River',
 'Klickitat Trail',
 'Indian Springs',
 'Weldon Wagon Rd Trail',
 'Lower Deschutes',
 'Cherry Orchard FCG trail',
 'Memaloose trail']

Trail popularity 1 year pre-fire (Sept 2016 - Sept 2017)

In [ ]:
start_date = '2016-09-15'
end_date = '2017-08-15'

test_n1 = v[ (v['d2p'] >= start_date) & (v['d2p'] <= end_date) ].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
test_n1 = test_n1.sort_values(by='tot_vis', ascending=False)
test_n1rank = test_n1['Name'].tolist()

test_n1rank

['Oneonta Gorge',
 'Munra Falls',
 'Latourell Falls',
 'Angels rest trail',
 'Eagle Creek',
 'Horsetail Falls',
 'Dog Mountain/Augspurger Mtn',
 'Wahkeena Falls',
 'Beacon Rock',
 'Umbrella/Sahalie Falls',
 'Nesmith Point, McCord, Elowa Falls',
 'Tamanawas Falls trail',
 'Cape Horn',
 'Surveyors ridge/cooks meadow',
 'Falls Creek Falls',
 'Tom McCall TNC trail',
 'Washougal River area',
 'Wind Mountain',
 'Mitchell/Wygant area',
 'Larch Mountain',
 'Coyote Wall area',
 'Mosier Twin Tunnels Trail',
 'Herman Creek',
 'Mosier Plateau FCG trail',
 'Spirit falls',
 'Starvation Creek area',
 'Ruckel Creek',
 'Catherine Creek area',
 'Panther Creek Falls',
 'Horsetail Creek',
 'Eight Mile trail',
 'Dog Creek Falls',
 'Tanner Butte',
 'Dog River',
 'Indian Springs',
 'Weldon Wagon Rd Trail',
 'Klickitat Trail',
 'Crawford Oaks WDNR',
 'Lower Deschutes',
 'Cherry Orchard FCG trail',
 'Memaloose trail']

Trail popularity 1 year post-fire (Sept 2017 - Sept 2018)

In [ ]:
start_date = '2017-09-15'
end_date = '2018-08-15'

test_p1 = v[ (v['d2p'] >= start_date) & (v['d2p'] <= end_date) ].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
test_p1 = test_p1.sort_values(by='tot_vis', ascending=False)
test_p1rank = test_p1['Name'].tolist()

test_p1rank

['Latourell Falls',
 'Beacon Rock',
 'Dog Mountain/Augspurger Mtn',
 'Umbrella/Sahalie Falls',
 'Falls Creek Falls',
 'Tamanawas Falls trail',
 'Tom McCall TNC trail',
 'Cape Horn',
 'Surveyors ridge/cooks meadow',
 'Washougal River area',
 'Wind Mountain',
 'Mitchell/Wygant area',
 'Spirit falls',
 'Coyote Wall area',
 'Mosier Twin Tunnels Trail',
 'Herman Creek',
 'Dog Creek Falls',
 'Weldon Wagon Rd Trail',
 'Starvation Creek area',
 'Catherine Creek area',
 'Cherry Orchard FCG trail',
 'Eight Mile trail',
 'Dog River',
 'Mosier Plateau FCG trail',
 'Panther Creek Falls',
 'Crawford Oaks WDNR',
 'Lower Deschutes',
 'Memaloose trail',
 'Klickitat Trail',
 'Oneonta Gorge',
 'Nesmith Point, McCord, Elowa Falls',
 'Ruckel Creek',
 'Munra Falls',
 'Larch Mountain',
 'Tanner Butte',
 'Indian Springs',
 'Horsetail Falls',
 'Wahkeena Falls',
 'Horsetail Creek',
 'Eagle Creek',
 'Angels rest trail']

Trail popularity 2 years post-fire (Sept 2018 - Sept 2019)

In [ ]:
start_date = '2018-09-15'
end_date = '2019-08-15'

test_p2 = v[ (v['d2p'] >= start_date) & (v['d2p'] <= end_date) ].groupby(['Name','burned'], as_index=False)['tot_vis'].sum()
test_p2 = test_p2.sort_values(by='tot_vis', ascending=False)
test_p2rank = test_p2['Name'].tolist()

test_p2rank

['Angels rest trail',
 'Latourell Falls',
 'Beacon Rock',
 'Dog Mountain/Augspurger Mtn',
 'Wahkeena Falls',
 'Tamanawas Falls trail',
 'Tom McCall TNC trail',
 'Umbrella/Sahalie Falls',
 'Horsetail Falls',
 'Falls Creek Falls',
 'Cape Horn',
 'Coyote Wall area',
 'Surveyors ridge/cooks meadow',
 'Washougal River area',
 'Mitchell/Wygant area',
 'Wind Mountain',
 'Larch Mountain',
 'Spirit falls',
 'Herman Creek',
 'Starvation Creek area',
 'Mosier Twin Tunnels Trail',
 'Panther Creek Falls',
 'Crawford Oaks WDNR',
 'Munra Falls',
 'Catherine Creek area',
 'Cherry Orchard FCG trail',
 'Eight Mile trail',
 'Klickitat Trail',
 'Dog Creek Falls',
 'Weldon Wagon Rd Trail',
 'Mosier Plateau FCG trail',
 'Dog River',
 'Memaloose trail',
 'Lower Deschutes',
 'Tanner Butte',
 'Eagle Creek',
 'Ruckel Creek',
 'Oneonta Gorge',
 'Nesmith Point, McCord, Elowa Falls',
 'Horsetail Creek',
 'Indian Springs']

Static plot of trail popularity pre and post fire years

In [ ]:
chart_n3 = alt.Chart(test_n3[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors', scale=alt.Scale(domain=[0, 80000]) ),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='3 years pre-fire (Sep 2014 - Aug 2015)',
    width=150,
    height=200,
)

chart_n2 = alt.Chart(test_n2[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors', scale=alt.Scale(domain=[0, 80000])),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2 years pre-fire (Sep 2015 - Aug 2016)',
    width=150,
    height=200,
)

chart_n1 = alt.Chart(test_n1[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors', scale=alt.Scale(domain=[0, 80000])),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='1 year pre-fire (Sep 2016 - Aug 2017)',
    width=150,
    height=200,
)

chart_p1 = alt.Chart(test_p1[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors', scale=alt.Scale(domain=[0, 80000])),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='1 year post-fire (Sep 2017 - Aug 2018)',
    width=150,
    height=200,
)

chart_p2 = alt.Chart(test_p2[0:10]).mark_bar(opacity=0.6).encode(
    y=alt.Y('Name:N', title = '', sort='-x'),
    x=alt.X('tot_vis:Q', title = 'Total visitors', scale=alt.Scale(domain=[0, 80000])),
    color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2 years post-fire (Sep 2018 - Aug 2019)',
    width=150,
    height=200,
)


trail_popularity2 = chart_n3 & chart_p1 | chart_n2 & chart_p2| chart_n1 
trail_popularity2

alt.HConcatChart(...)

# Show total visitors before and after fire


In [ ]:
chart_p2 = alt.Chart(test_p2).mark_line(opacity=0.6).encode(
    x=alt.X('monthdate(d2p):O'),
    y=alt.Y('sum(tot_vis):Q', title = 'Total visitors', scale=alt.Scale(domain=[0, 620000])),
    #color = alt.Color('burned:N', scale=alt.Scale(range=colorCustom))
).properties(
    title='2 years post-fire (Sep 2018 - Aug 2019)',
    width=150,
    height=200,
)

chart_p2

alt.Chart(...)

In [ ]:
chart = alt.Chart(v).mark_line().encode(
    x = alt.X("d2p:O", timeUnit="yearmonth", title="Date"),
    y=alt.Y("sum(tot_vis):Q",title="Total Visitors"),
    #color=alt.Color("siteid:N")
    #tooltip=['tot_vis']
).properties(
    title="Overall Trail Visitors (Sep 2014 - Aug 2019)",
    width=600,
    height=150,
)

# Mark's when Eagle Creek Fire happened
fire_rule = alt.Chart(v).mark_rule(
        color="orange", 
        strokeWidth=5
    ).encode(
        x="d2p",
    ).transform_filter("datum.year == 2017 && datum.month == 9")

fire_text = alt.Chart(v).mark_text(
    text = 'Eagle Creek Fire', 
    angle=270,
    fontSize=12
).encode(
        x="d2p",
    ).transform_filter("datum.year == 2017 && datum.month == 10")

# Denote each year
year_rule = alt.Chart(v).mark_rule(
        color='black',
        strokeDash=[3,5]
    ).encode(
        x="d2p",
    ).transform_filter("[datum.year == 2015 || datum.year == 2016 || datum.year == 2017 || datum.year == 2018 || datum.year == 2019] && datum.month == 1")
    

total_vistors = chart + fire_rule + fire_text + year_rule    
total_vistors

alt.LayerChart(...)

In [ ]:
total_vistors.save('total_vistors.json')

In [ ]:
total_vistors.save('total_vistors.json')

from google.colab import files
files.download('total_vistors.json') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>